In [2]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

# Dados do problema (use dados_sit_1_2 para exemplo)
estratos = ['E1', 'E2', 'E3', 'E4', 'E5']

dados = {
    'area_max': {'E1': 21.66, 'E2': 14.17, 'E3': 11.33, 'E4': 6.34, 'E5': 7.53},
    'renda_liquida': {'E1': 404.40, 'E2': 286.50, 'E3': 183.30, 'E4': 102.57, 'E5': 127.00},
    'rend_lenha': {'E1': 50.57, 'E2': 53.08, 'E3': 43.09, 'E4': 40.71, 'E5': 38.69},
    'rend_estacas': {'E1': 0, 'E2': 0, 'E3': 0, 'E4': 0, 'E5': 0},  # Vazio no Sit_1_2, pode ajustar se quiser
    'mao_de_obra': {'E1': 67.5, 'E2': 68.3, 'E3': 58.0, 'E4': 56.5, 'E5': 48.6}
}

# Sensibilidade ambiental (p_j)
sensibilidade = {'E1': 1.0, 'E2': 0.9, 'E3': 0.6, 'E4': 0.5, 'E5': 0.4}

# Parâmetros das restrições
restricoes = {
    'sl_min': 2300,               # lenha mínima
    'mo_max': 3500,               # mão de obra máxima
    'em_min': 0,                 # estacas mínimas (zero para Sit_1_2; ajustar se for Sit_3_4 ou 5)
    'area_sensivel_max': 25,     # área max para estratos sensíveis E1,E2
    'area_total_fazenda': 460,   # área total da fazenda
    'prop_preservacao': 0.2      # preservação 20% da fazenda
}



def modelo_multiobjetivo(dados, sensibilidade, restricoes, lambda_val, var_type='Integer'):
    model = gp.Model("Planejamento_Florestal_Multiobjetivo")
    model.Params.OutputFlag = 1  # Mostrar log (pode zeroar para silêncio)

    cat = GRB.INTEGER if var_type == 'Integer' else GRB.CONTINUOUS

    # Variáveis inteiras para as áreas exploradas
    E = model.addVars(estratos, lb=0, vtype=cat, name="Area")

    # Objetivo econômico
    z_econ = gp.quicksum(dados['renda_liquida'][j] * E[j] for j in estratos)

    # Objetivo ambiental
    z_ambiental = gp.quicksum(sensibilidade[j] * E[j] for j in estratos)

    # Função objetivo ponderada
    model.setObjective(lambda_val * z_econ - (1 - lambda_val) * z_ambiental, GRB.MAXIMIZE)

    # Restrições

    # Área máxima por estrato
    for j in estratos:
        model.addConstr(E[j] <= dados['area_max'][j], name=f"Limite_Area_{j}")

    # Lenha mínima
    if 'sl_min' in restricoes and restricoes['sl_min'] > 0:
        model.addConstr(
            gp.quicksum(dados.get('rend_lenha', {}).get(j, 0) * E[j] for j in estratos) >= restricoes['sl_min'],
            name="Lenha_Min"
        )

    # Mão de obra máxima
    if 'mo_max' in restricoes and restricoes['mo_max'] > 0:
        model.addConstr(
            gp.quicksum(dados.get('mao_de_obra', {}).get(j, 0) * E[j] for j in estratos) <= restricoes['mo_max'],
            name="Mao_Obra_Max"
        )

    # Estacas mínimas (se definido e > 0)
    if 'em_min' in restricoes and restricoes['em_min'] > 0:
        model.addConstr(
            gp.quicksum(dados.get('rend_estacas', {}).get(j, 0) * E[j] for j in estratos) >= restricoes['em_min'],
            name="Estacas_Min"
        )

    # Área sensível máxima (E1 + E2)
    if 'area_sensivel_max' in restricoes and restricoes['area_sensivel_max'] > 0:
        model.addConstr(
            E['E1'] + E['E2'] <= restricoes['area_sensivel_max'],
            name="Area_Sensivel_Max"
        )

    # Área total máxima (respeitando preservação)
    if ('area_total_fazenda' in restricoes and restricoes['area_total_fazenda'] > 0 and
        'prop_preservacao' in restricoes):
        max_area_utilizavel = restricoes['area_total_fazenda'] * (1 - restricoes['prop_preservacao'])
        model.addConstr(
            gp.quicksum(E[j] for j in estratos) <= max_area_utilizavel,
            name="Area_Total_Max"
        )

    # Otimizar
    model.optimize()

    # Resultado
    if model.Status == GRB.OPTIMAL:
        print(f"\nSolução Ótima para lambda = {lambda_val:.2f}")
        print(f"Valor da função objetivo: {model.ObjVal:.2f}")
        print("Áreas a explorar por estrato (hectares):")
        for j in estratos:
            print(f"   {j}: {E[j].X:.2f}")
        print(f"Valor do objetivo econômico: {z_econ.getValue():.2f}")
        print(f"Valor do objetivo ambiental: {z_ambiental.getValue():.2f}")
    else:
        print("Modelo não encontrou solução ótima.")
        if model.Status == GRB.INFEASIBLE:
            print("Modelo inviável!")
            model.computeIIS()
            print("Restrições inviabilizantes:")
            for c in model.getConstrs():
                if c.IISConstr:
                    print(f" - {c.ConstrName}")
    return model, E

# --- Execução do modelo ---

modelo, variaveis = modelo_multiobjetivo(dados, sensibilidade, restricoes, lambda_val=0.4, var_type='Continuous')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 112.1 MB/s eta 0:00:00
Restricted license - for non-production use only - expires 2026-11-23
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 5 columns and 22 nonzeros
Model fingerprint: 0x6b2fa394
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [4e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 4e+03]
Presolve removed 6 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.2024461e+03   2.153082e+02   0.000000e+00      0s
       3    5.3372551e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work unit